In [1]:
# start time - 1609459200000, 1641028611000, 1672564611000, 1654045261000, 1651366861000 (2021, 2022, 2023)

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import ta
import datetime as dt
import numpy as np
import time
import itertools
import os

In [4]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

In [5]:
# start_time_arr = [1609459200000]

interval_arr = ['15m', '1h']

sl_atr_arr = [0.5, 1, 2, 3, 4]

tp_atr_arr = [3, 5, 7, 9, 12]

# stop loss determine candlestick
sl_det_arr = ['Close']

In [6]:
timezone = 8
# symbol = 'ethusdt'
symbol_arr = ['nearusdt', 'apeusdt', 'avaxusdt', 'adausdt']

start_time_d = 1641028611000
end_time = round(time.time() * 1000)
# end_time = 1654060250000 
# step between timestamps in milliseconds
step = 1000

In [7]:
dataframes = {}
def create_raw(symbol_arr, interval_arr, start_time, end_time, step):
        
    for interval in interval_arr:
        for symbol in symbol_arr:
            # Fetch the data using batch requests
            data = []    
            while start_time < end_time:
                limit = min(step, end_time - start_time + 1)  # Adjust the limit for the last batch
                response = client.get_klines(symbol=symbol.upper(), interval=interval, limit=limit, startTime=start_time)

                if len(response) == 0:
                    break  # No more data available, exit the loop
                data.extend(response)
                start_time = response[-1][0] + 1

            # Convert the data to a DataFrame
            columns = [
                "Open_Time", "Open", "High", "Low", "Close", "Volume", "Close_Time",
                "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                "Taker buy quote asset volume", "Ignore"
            ]
            raw_df = pd.DataFrame(data, columns=columns)   

            raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

            dataframes[f'df_{interval}_{symbol}'] = raw_df
            start_time = start_time_d

In [8]:
loop_start_time = time.time()
create_raw(symbol_arr, interval_arr, start_time_d, end_time, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")
print(dataframes)

Time taken to execute for loop: 35.3983850479126 seconds
{'df_15m_nearusdt':            Open_Time         Open        Close         High          Low  \
0      1641029400000  14.66200000  14.58400000  14.67700000  14.56300000   
1      1641030300000  14.58300000  14.62800000  14.62800000  14.56100000   
2      1641031200000  14.62600000  14.65300000  14.71300000  14.60800000   
3      1641032100000  14.65100000  14.52800000  14.65300000  14.52400000   
4      1641033000000  14.53200000  14.58400000  14.59900000  14.44800000   
...              ...          ...          ...          ...          ...   
49690  1685754900000   1.60600000   1.60400000   1.60800000   1.60200000   
49691  1685755800000   1.60300000   1.60100000   1.60500000   1.59800000   
49692  1685756700000   1.60200000   1.59900000   1.60200000   1.59800000   
49693  1685757600000   1.59900000   1.60700000   1.60800000   1.59900000   
49694  1685758500000   1.60600000   1.61100000   1.61100000   1.60500000   

          

In [9]:
def get_klines(symbol, interval, start_time, end_time):
    global dataframes
    df = dataframes[f'df_{interval}_{symbol}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [10]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df):

# ema
    for i in (200, 500, 1000):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
#     rsi_int = 14
#     df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()

# kd
#     df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
#     df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
#     df['kd_cross'] = check_cross(df, kd_dir)

In [11]:
def conditions(df):
    df['c1'] = df['Low'] <= df['bb_l']
    df['c2'] = df['ema_200'] >= df['ema_500']
    df['c3'] = df['ema_500'] >= df['ema_1000']
    df['c4'] = df['Volume'] >= df['Volume'].shift(1)
    df['c5'] = df['Close'] >= df['Open']
    df['c6'] = df['Close'].shift(1) <= df['Open'].shift(1)
    df['c7'] = df['Low'] <= df['ema_500']
    
    # signal
    df['signal'] = df['c1'] & df['c2'] & df['c3'] & df['c4'] & df['c5'] & df['c6'] & df['c7']
    
    # open_entry
    df['open_entry'] = df['signal'].shift()
    
    return df

In [12]:
def entries(df, sl_atr, tp_atr, sl_det):

    in_position = False
    stop_loss = np.nan
    take_profit = np.nan
    close_val = df['Close']
    atr_val = df['atr']

    df['position'] = ''  # Create an empty column for position

    for index, row in df.iterrows():

        if index == 0:
            continue

        elif row['open_entry']:
            df.at[index, 'entry_p'] = close_val.shift(1).at[index]
            df.at[index, 'stop_loss'] = close_val.shift(1).at[index] - sl_atr * atr_val.shift(1).at[index]
            df.at[index, 'take_profit'] = close_val.shift(1).at[index] + tp_atr * atr_val.shift(1).at[index]
            df.at[index, 'position'] = 'Buy'
            in_position = True
            stop_loss = df.at[index, 'stop_loss']
            take_profit = df.at[index, 'take_profit']

        elif in_position and row['Close'] <= stop_loss:
            df.at[index, 'position'] = 'Stop'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan

        elif in_position and row['High'] >= take_profit:
            df.at[index, 'position'] = 'Sell'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan
            
    df = df[(df['open_entry']) | (df['position'] != '')]

In [13]:
# 部位回測
def backtest(df):

    df = df.reset_index(drop = True)
    df = df[(df['position'] == 'Buy') |
                  (df['position'] == 'Sell') |
                  (df['position'] == 'Stop')]

    # 一次進場多少單位
    pos_size = 1

    col = ['Open_Time', 'Open', 'Close', 'High', 'Low', 'atr', 'position','entry_p', 'stop_loss', 'take_profit']
    pos = df[col]
    pos = pos.reset_index(drop = True)


    for index, row in pos.iterrows():

        current_pos = 0

        # 進場
        if pos.at[index, 'position'] == 'Buy':
            pos.at[index, 'size'] = pos_size
            pos.exit_p = np.nan

        # 出場
        if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':

            # 停利：達成條件時收盤價
            if pos.at[index, 'position'] == 'Sell':
                for i in range(index -1, -1, -1):
                    if pos.at[i, 'position'] == 'Buy':
                        pos.at[index, 'exit_p'] = pos.at[i, 'take_profit']
                    break

            # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
            if pos.at[index, 'position'] == 'Stop':
                pos.at[index, 'exit_p'] = pos.at[index, 'Open']
#                 for i in range(index -1, -1, -1):
#                     if pos.at[i, 'position'] == 'Buy':
#                         pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
#                     break

            # 計算每次出場部位大小（每次出場皆清倉）
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    current_pos += pos.at[i, 'size']
                    if i == 0:
                        pos.at[index, 'size'] = -current_pos
                    else:
                        continue
                else:
                    pos.at[index, 'size'] = -current_pos
                    current_pos = 0
                    break


    # 計算部位價值
    for index, row in pos.iterrows():
        if pos.at[index, 'position'] == 'Buy':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
        elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


    # 若最後一筆為 Buy，移除該單，迭代驗證
    for index, row in pos.iloc[::-1].iterrows():
        if row['position'] == 'Buy':
            pos = pos.drop(index)
        else:
            break


    # 手續費、滑點、價差
    fee = 0.05 / 100
    amt_abs_sum = pos.amt.abs().sum()
    ttl_fee = amt_abs_sum * fee

    # 損益
    leverage = 10
    ttl_profit = -pos.amt.sum() - ttl_fee

    
    # 計算獲利/虧損次數
    agg_amts = []

    for i in range(len(pos) - 1, -1, -1):

        if pos.loc[i, 'position'] in ['Stop', 'Sell']:

            # look out for the + sign
            total_amt = pos.loc[i, 'amt'] + np.absolute(pos.loc[i, 'amt']) * fee

            # iterate backwards from the current row until reaching another 'Stop' or 'Sell'
            # watch out for the + in total_amt += trading_fee
            j = i - 1
            while j >= 0 and pos.loc[j, 'position'] not in ['Stop', 'Sell']:
                total_amt += pos.loc[j, 'amt']
                trading_fee = np.absolute(pos.loc[j, 'amt']) * fee
                total_amt += trading_fee
                j -= 1

            # add the aggregated amount to the list
            agg_amts.append(total_amt)

    agg_amts.reverse()

    
    # 計算進場最大部位，最大損益
    consec_entry = 0
    position_amt_sum = 0
    max_consec_entry = 0
    max_position = 0
    max_profit = 0
    max_loss = 0

    for index, row in pos.iterrows():

        if row['position'] == 'Buy':

            consec_entry += 1
            position_amt_sum += row['amt']

        elif row['position'] in ['Sell', 'Stop']:

            if consec_entry > max_consec_entry:
                max_consec_entry = consec_entry
                max_position = position_amt_sum

            position_amt_sum += row['amt']

            if -position_amt_sum > max_profit:
                max_profit = -position_amt_sum

            if -position_amt_sum < max_loss:
                max_loss = -position_amt_sum

            consec_entry = 0
            position_amt_sum = 0

        else:
            pass


    # 最大部位
    profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)


    # 勝率
    wins = 0
    loses = 0

    for trade in agg_amts:
        if trade < 0:
            wins += 1
        elif trade > 0:
            loses += 1

    win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)

    cumulative_values = []
    cumulative_sum = 0

    for value in agg_amts:
        cumulative_sum -= value
        cumulative_values.append(cumulative_sum)

    max_drawdown = min(cumulative_values)
    
    # 結果
    result = {'Profit': [round(ttl_profit, 2)],
              'Fee': [round(ttl_fee, 2)],
              'Max_Profit': [round(max_profit, 2)],
              'Max_Loss': [round(max_loss, 2)],
              'Max_Entry': [max_consec_entry],
              'Max_Position': [round(max_position, 2)],
              'Profit_%': [profit_per],
              'Win_Rate': [win_rate],
              'PF_Ratio': [round(ttl_profit/ttl_fee, 2)],
              'Max_Drawdown': [round(max_drawdown,2)]}


    result_df = pd.DataFrame(result)

    return result_df, ttl_profit, win_rate, ttl_fee

In [14]:
# iterate using start_time, under same time interval, test different variables
def run_backtest():

    results_df = pd.DataFrame()
    
    symbol_list = []
    interval_list = []
    sl_atr_list = []
    tp_atr_list = []
    sl_det_list = []

    i = 0

    loop_start_time = time.time()
    
    parameters = [(symbol, interval, sl_atr, tp_atr, sl_det) 
                  for symbol in symbol_arr
                  for interval in interval_arr
                  for sl_atr in sl_atr_arr
                  for tp_atr in tp_atr_arr
                  for sl_det in sl_det_arr]
    
    for symbol, interval, sl_atr, tp_atr, sl_det in parameters:

        df = get_klines(symbol, interval, start_time_d, end_time)
        df = df.reset_index()
        indicators(df)
        conditions(df)
        entries(df, sl_atr, tp_atr, sl_det)
        backtest_df, ttl_profit, win_rate, fee = backtest(df)

        results_df = pd.concat([results_df, backtest_df], ignore_index = True)

        symbol_list.append(symbol)
        interval_list.append(interval)
        sl_atr_list.append(sl_atr)
        tp_atr_list.append(tp_atr)
        sl_det_list.append(sl_det)

        i += 1
        
        print(f" {i} - {symbol}, {interval}, sl: {sl_atr}, tp: {tp_atr}, {sl_det}, {round(ttl_profit, 2)}, {round(fee, 2)}, {round(ttl_profit/fee, 2)}, {win_rate} ")

    loop_end_time = time.time()

    print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

    results_df['symbol'] = symbol_list
    results_df['interval'] = interval_list
    results_df['sl_atr'] = sl_atr_list
    results_df['tp_atr'] = tp_atr_list
    results_df['sl_det'] = sl_det_list

    results_df = results_df[['symbol', 'interval',
                             'sl_atr', 'tp_atr', 'sl_det',
                             'Profit', 'Fee', 'Max_Profit', 'Max_Loss',
                             'Max_Entry', 'Max_Position', 'Profit_%', 'Win_Rate', 'PF_Ratio', 'Max_Drawdown'
                            ]]
    
    return results_df

In [15]:
results_df = run_backtest()
results_df = results_df.sort_values('Profit', ascending = False)
# print(results_df)

 1 - nearusdt, 15m, sl: 0.5, tp: 3, Close, 0.9, 0.41, 2.18, 34.38% 
 2 - nearusdt, 15m, sl: 0.5, tp: 5, Close, 0.56, 0.41, 1.35, 34.38% 
 3 - nearusdt, 15m, sl: 0.5, tp: 7, Close, 0.05, 0.41, 0.12, 32.81% 
 4 - nearusdt, 15m, sl: 0.5, tp: 9, Close, 0.3, 0.41, 0.74, 32.81% 
 5 - nearusdt, 15m, sl: 0.5, tp: 12, Close, 0.38, 0.41, 0.92, 31.25% 
 6 - nearusdt, 15m, sl: 1, tp: 3, Close, 1.97, 0.41, 4.78, 36.67% 
 7 - nearusdt, 15m, sl: 1, tp: 5, Close, 2.32, 0.41, 5.64, 35.00% 
 8 - nearusdt, 15m, sl: 1, tp: 7, Close, 2.63, 0.41, 6.37, 30.00% 
 9 - nearusdt, 15m, sl: 1, tp: 9, Close, 3.57, 0.41, 8.65, 28.33% 
 10 - nearusdt, 15m, sl: 1, tp: 12, Close, 5.67, 0.41, 13.7, 25.42% 
 11 - nearusdt, 15m, sl: 2, tp: 3, Close, 1.17, 0.41, 2.84, 42.59% 
 12 - nearusdt, 15m, sl: 2, tp: 5, Close, 1.3, 0.41, 3.15, 35.29% 
 13 - nearusdt, 15m, sl: 2, tp: 7, Close, 1.97, 0.41, 4.78, 29.41% 
 14 - nearusdt, 15m, sl: 2, tp: 9, Close, 2.4, 0.41, 5.82, 28.00% 
 15 - nearusdt, 15m, sl: 2, tp: 12, Close, 5.69, 

 121 - avaxusdt, 15m, sl: 4, tp: 3, Close, -9.05, 2.82, -3.21, 57.45% 
 122 - avaxusdt, 15m, sl: 4, tp: 5, Close, -4.46, 2.82, -1.58, 45.45% 
 123 - avaxusdt, 15m, sl: 4, tp: 7, Close, 1.87, 2.82, 0.66, 34.21% 
 124 - avaxusdt, 15m, sl: 4, tp: 9, Close, 14.96, 2.83, 5.29, 28.57% 
 125 - avaxusdt, 15m, sl: 4, tp: 12, Close, 40.94, 2.84, 14.41, 26.47% 
 126 - avaxusdt, 1h, sl: 0.5, tp: 3, Close, -0.37, 0.22, -1.63, 50.00% 
 127 - avaxusdt, 1h, sl: 0.5, tp: 5, Close, -0.19, 0.22, -0.83, 50.00% 
 128 - avaxusdt, 1h, sl: 0.5, tp: 7, Close, -0.01, 0.22, -0.03, 50.00% 
 129 - avaxusdt, 1h, sl: 0.5, tp: 9, Close, 0.17, 0.22, 0.77, 50.00% 
 130 - avaxusdt, 1h, sl: 0.5, tp: 12, Close, 0.44, 0.22, 1.97, 50.00% 
 131 - avaxusdt, 1h, sl: 1, tp: 3, Close, -2.09, 0.22, -9.4, 28.57% 
 132 - avaxusdt, 1h, sl: 1, tp: 5, Close, -1.92, 0.22, -8.59, 28.57% 
 133 - avaxusdt, 1h, sl: 1, tp: 7, Close, -1.74, 0.22, -7.79, 28.57% 
 134 - avaxusdt, 1h, sl: 1, tp: 9, Close, -1.56, 0.22, -6.98, 28.57% 
 135 - avax

In [16]:
results_df.to_csv(f'results_df_021.csv')